In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import pickle

In [2]:
data = pd.read_csv('Churn_Modelling.csv')
data


,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9996,15606229,Obijiaku,771,France,Male,39,5,0.00,2,1,0,96270.64,0
9996,9997,15569892,Johnstone,516,France,Male,35,10,57369.61,1,1,1,101699.77,0
9997,9998,15584532,Liu,709,France,Female,36,7,0.00,1,0,1,42085.58,1
9998,9999,15682355,Sabbatini,772,Germany,Male,42,3,75075.31,2,1,0,92888.52,1


In [3]:
# Preprocess the data
# Drop irrelevant columns

data = data.drop(columns=['RowNumber', 'CustomerId', 'Surname'])
print('Shape of the data after dropping columns:', data.shape)
data.head()

Shape of the data after dropping columns: (10000, 11)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
# Encode categorical data

data['Gender'].value_counts()

Gender
Male      5457
Female    4543
Name: count, dtype: int64

In [5]:
label_encoder = LabelEncoder()
data['Gender'] = label_encoder.fit_transform(data['Gender'])
data['Gender'].value_counts()

Gender
1    5457
0    4543
Name: count, dtype: int64

In [6]:
data['Geography'].value_counts()

# We won't use label encoding for this dataset because Geography has  multiple values and each value will have 1,2,3 like this. The model can think 3 has higher value than 1 and 2 and vice versa.

Geography
France     5014
Germany    2509
Spain      2477
Name: count, dtype: int64

In [7]:
# One-hot encoding for Geography column

from sklearn.preprocessing import OneHotEncoder
one_hot_encoder = OneHotEncoder()

one_hot_encoded_data = one_hot_encoder.fit_transform(data[['Geography']])

In [8]:
one_hot_encoded_data.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [9]:
one_hot_encoder.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [10]:
geo_one_hot_encoded = pd.DataFrame(data = one_hot_encoded_data.toarray(), columns=one_hot_encoder.get_feature_names_out(['Geography']))
geo_one_hot_encoded.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [11]:
# Combine one hot encoded columns with original data

data = pd.concat([data.drop(columns=['Geography']), geo_one_hot_encoded], axis=1)

data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [12]:
X = data.drop(['EstimatedSalary'], axis=1)
y = data['EstimatedSalary']

# Splitting the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

print('Xtrain shape:', X_train.shape)
print('Xtest shape:', X_test.shape)
print('ytrain shape:', y_train.shape)
print('ytest shape:', y_test.shape)

Xtrain shape: (8000, 12)
Xtest shape: (2000, 12)
ytrain shape: (8000,)
ytest shape: (2000,)


In [13]:
## Scale these features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [15]:
X_test

array([[-0.55204276, -1.09168714, -0.36890377, ..., -1.01460667,
         1.75486502, -0.57369368],
       [-1.31490297, -1.09168714,  0.10961719, ...,  0.98560362,
        -0.5698444 , -0.57369368],
       [ 0.57162971, -1.09168714,  0.30102557, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       ...,
       [-0.74791227,  0.91601335, -0.27319958, ..., -1.01460667,
        -0.5698444 ,  1.74309049],
       [-0.00566991,  0.91601335, -0.46460796, ..., -1.01460667,
         1.75486502, -0.57369368],
       [-0.79945688,  0.91601335, -0.84742473, ..., -1.01460667,
         1.75486502, -0.57369368]])

In [16]:
# Save the encoders and scalers

with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder, file)

with open('onehot_encoder_geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder, file)

# Save scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Regression Problem Statement

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [19]:
# Build ANN model

model = Sequential(
    [
        Dense(64, activation="relu", input_shape=(X_train.shape[1],)),  # HL1 Connected with input layet
        Dense(32, activation="relu"),  # HL2 Connected with HL1
        Dense(1) # Output layer, by default it uses linear activation function  
    ]
)

In [21]:
# Compile the model

model.compile(optimizer='adam', loss='mean_absolute_error', metrics=['mae'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [23]:
# Setup the Tensorboard

log_dir = "regressionlogs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [24]:
# Setup Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

In [25]:
# Train the model

history = model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=100,
                    callbacks=[tensorflow_callback, early_stopping_callback])

Epoch 1/100
250/250 [==============================] - 1s 1ms/step - loss: 100106.7969 - mae: 100106.7969 - val_loss: 99511.3594 - val_mae: 99511.3594
Epoch 2/100
250/250 [==============================] - 0s 973us/step - loss: 99234.8516 - mae: 99234.8516 - val_loss: 97767.1328 - val_mae: 97767.1328
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 96238.4141 - mae: 96238.4141 - val_loss: 93400.6875 - val_mae: 93400.6875
Epoch 4/100
250/250 [==============================] - 0s 937us/step - loss: 90435.4844 - mae: 90435.4844 - val_loss: 86210.7969 - val_mae: 86210.7969
Epoch 5/100
250/250 [==============================] - 0s 903us/step - loss: 82129.4922 - mae: 82129.4922 - val_loss: 77038.0547 - val_mae: 77038.0547
Epoch 6/100
250/250 [==============================] - 0s 866us/step - loss: 72626.0625 - mae: 72626.0625 - val_loss: 67741.9844 - val_mae: 67741.9844
Epoch 7/100
250/250 [==============================] - 0s 920us/step - loss: 63819.0469 - mae: 6

In [26]:
# Load Tensorboard Extension
%load_ext tensorboard

In [28]:
%tensorboard --logdir regressionlogs/fit

Reusing TensorBoard on port 6006 (pid 33644), started 0:00:29 ago. (Use '!kill 33644' to kill it.)

In [29]:
model.save('model.h5')

/Users/nkejriwal/Documents/Learning/DL_ANN/learn_env/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [30]:
# Evaluate the model based on the test data
test_loss, test_mae = model.evaluate(X_test, y_test)

print(f'Test MAE: {test_mae:.2f}')

63/63 [==============================] - 0s 433us/step - loss: 49827.3320 - mae: 49827.3320
Test MAE: 49827.33


In [31]:
X

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,1,0.0,1.0,0.0


In [32]:
X['Exited'].value_counts()

Exited
0    7963
1    2037
Name: count, dtype: int64